# Python学习笔记
使用JupyterNotebook的主要考虑是因为兼顾展示跟运行。

## 数据结构
最近开始学习一些基本数据结构，并尝试用Python实现一下。

### 栈
Python的list天生就是实现栈的好基础，拥有`O(1)`的`push`, `pop`操作。

先来写一个测试：

In [33]:
# creating a stack is trivial
stack = Stack()
# initialized as a empty instance
assert True == stack.is_empty()
# push one test value
stack.push(1)
# now it should NOT be empty
assert False == stack.is_empty()
# peek the value on the top
assert 1 == stack.top()
# top should not change the stack
assert False == stack.is_empty()
# push another value
stack.push(2)
# peek the value on the top
assert 2 == stack.top()
# pop the value from the top
assert 2 == stack.pop()
# now the value on the top should be 1
assert 1 == stack.top()
assert 1 == stack.pop()
assert True == stack.is_empty()


现在我们来添加一个实现：

In [32]:
class Stack:

    def __init__(self):
        self.data = []

    def push(self, item):
        self.data.append(item)

    def pop(self):
        return self.data.pop()

    def top(self):
        return self.data[-1]

    def is_empty(self):
        return len(self.data) == 0

**练习** 你也可以自己写一个实现来覆盖上面这个参考实现：

In [30]:
class Stack:

    def __init__(self):
        '''
        >>> stack = Stack()
        >>> stack.push(1)
        >>> stack.push(2)
        >>> stack.pop()
        2
        >>> stack.pop()
        1
        '''
        raise NotImplementedError()

    def push(self, item):
        '''
        >>> stack = Stack()
        >>> stack.push(1)
        '''
        raise NotImplementedError()
        
    def pop(self):
        '''
        >>> stack = Stack()
        >>> stack.push(1)
        >>> stack.push(2)
        >>> stack.pop()
        2
        >>> stack.pop()
        1
        '''
        raise NotImplementedError()
        
    def top(self):
        '''
        >>> stack = Stack()
        >>> stack.push(1)
        >>> stack.top()
        1
        >>> stack.push(2)
        >>> stack.top()
        2
        >>> stack.pop()
        2
        >>> stack.top()
        1
        '''
        raise NotImplementedError()
        
    def is_empty(self):
        '''
        >>> stack = Stack()
        >>> stack.is_empty()
        True
        >>> stack.push(1)
        >>> stack.is_empty()
        False
        >>> stack.pop()
        1
        >>> stack.is_empty()
        True
        '''
        raise NotImplementedError()
        
import doctest
doctest.testmod(verbose=False)

TestResults(failed=0, attempted=25)

### 队列

#### 接口
有些教科书用`enqueue`, `dequeue`来从接口上区别队列与栈概念上的区别。但是刻意的模糊这个区别其实在各种算法替换上是有好处的。
尤其是后面我们实现`BFS`, `DFS`, `Greedy Search`, `A*`时，接口一致相当方便。

In [34]:
class Queue:

    def push(self, item):
        raise NotImplementedError()

    def pop(self):
        raise NotImplementedError()

    def is_empty(self):
        raise NotImplementedError()

#### 简单实现

In [2]:
class Queue:
    '''
    >>> queue = Queue()
    >>> queue.is_empty()
    True
    >>> queue.push(1)
    >>> queue.is_empty()
    False
    >>> queue.push(2)
    >>> queue.pop()
    1
    >>> queue.pop()
    2
    >>> queue.is_empty()
    True
    '''

    def __init__(self):
        self.data = []

    def push(self, item):
        self.data.append(item)

    def pop(self):
        return self.data.pop(0)

    def is_empty(self):
        return len(self.data) == 0

import doctest
doctest.testmod()

TestResults(failed=0, attempted=8)

这个实现的问题是，性能不太好。因为python内置的`list`在`pop(0)`时是`O(n)`的时间复杂度。我们可以来测量一下。
我们看看当连续调用`n`次`push`时总共耗时。

In [5]:
import timeit

def npush(n):
    queue = Queue()
    for i in range(n):
        queue.push(i)
    return queue

def npop(queue):
    while not queue.is_empty():
        queue.pop()

print('testing push:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npush(n)', setup=f'from __main__ import npush; n = {10**i}').timeit(number=1) / 10 ** i)


print('testing pop:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npop(queue)', setup=f'from __main__ import (npush, npop); queue = npush({10**i})').timeit(number=1) / 10 ** i)

testing push:
1 average: 7.353024557232857e-06
10 average: 7.20401294529438e-07
100 average: 5.439098458737135e-07
1000 average: 2.0830496214330197e-07
10000 average: 2.3680089507251978e-07
100000 average: 2.3715732037089766e-07
testing pop:
1 average: 5.2319373935461044e-06
10 average: 6.939051672816277e-07
100 average: 4.812201950699091e-07
1000 average: 5.012049805372954e-07
10000 average: 1.8304382101632654e-06
100000 average: 1.9037084210431204e-05


看起来符合预期，`pop`操作是`O(n)`的，测试时间是`O(n^2)`的。
看看另一个实现：

In [7]:
class Queue:
    '''
    >>> queue = Queue()
    >>> queue.is_empty()
    True
    >>> queue.push(1)
    >>> queue.is_empty()
    False
    >>> queue.push(2)
    >>> queue.pop()
    1
    >>> queue.pop()
    2
    >>> queue.is_empty()
    True
    '''

    def __init__(self):
        self.data = []

    def push(self, item):
        self.data.insert(0, item)

    def pop(self):
        return self.data.pop()

    def is_empty(self):
        return len(self.data) == 0

import doctest
doctest.testmod()

import timeit

def npush(n):
    queue = Queue()
    for i in range(n):
        queue.push(i)
    return queue

def npop(queue):
    while not queue.is_empty():
        queue.pop()

print('testing push:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npush(n)', setup=f'from __main__ import npush; n = {10**i}').timeit(number=1) / 10 ** i)


print('testing pop:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npop(queue)', setup=f'from __main__ import (npush, npop); queue = npush({10**i})').timeit(number=1) / 10 ** i)

testing push:
1 average: 9.363982826471329e-06
10 average: 8.736038580536842e-07
100 average: 5.306897219270468e-07
1000 average: 6.182799115777016e-07
10000 average: 3.22516089072451e-06
100000 average: 2.9823674709768967e-05
testing pop:
1 average: 4.5370543375611305e-06
10 average: 5.575013346970081e-07
100 average: 3.5595032386481764e-07
1000 average: 3.5154994111508133e-07
10000 average: 4.2142040329054e-07
100000 average: 3.7220199941657484e-07


有趣，看起来`list.append()`比`list.pop()`更快，`list.pop(0)`比`list.insert(0)`更快么？

现在写一个正常一点的`Queue`。

In [10]:
class Queue:
    '''
    >>> queue = Queue()
    >>> queue.is_empty()
    True
    >>> queue.push(1)
    >>> queue.is_empty()
    False
    >>> queue.push(2)
    >>> queue.pop()
    1
    >>> queue.pop()
    2
    >>> queue.is_empty()
    True
    '''

    class Node:
        def __init__(self, entry, next=None):
            self.entry = entry
            self.next = next

    def __init__(self):
        self.tail = None

    def push(self, item):
        if self.tail:
            self.tail.next = Queue.Node(item, self.tail.next)
            self.tail = self.tail.next
        else:
            self.tail = Queue.Node(item, None)
            self.tail.next = self.tail

    def pop(self):
        head = self.tail.next
        item = head.entry
        if head is self.tail:
            self.tail = None
        else:
            self.tail.next = head.next
        return item

    def is_empty(self):
        return self.tail is None

import doctest
doctest.testmod()

import timeit

def npush(n):
    queue = Queue()
    for i in range(n):
        queue.push(i)
    return queue

def npop(queue):
    while not queue.is_empty():
        queue.pop()

print('testing push:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npush(n)', setup=f'from __main__ import npush; n = {10**i}').timeit(number=1) / 10 ** i)


print('testing pop:')
for i in range(6):
    print(10 ** i, 'average:', timeit.Timer('npop(queue)', setup=f'from __main__ import (npush, npop); queue = npush({10**i})').timeit(number=1) / 10 ** i)

testing push:
1 average: 7.847091183066368e-06
10 average: 1.615402288734913e-06
100 average: 1.3355002738535405e-06
1000 average: 1.3227759627625345e-06
10000 average: 1.146089902613312e-06
100000 average: 7.93335639173165e-07
testing pop:
1 average: 4.116096533834934e-06
10 average: 7.504015229642391e-07
100 average: 5.687703378498554e-07
1000 average: 5.824069958180189e-07
10000 average: 6.088869995437563e-07
100000 average: 8.110076305456459e-07
